* Use the [MassMessage API](https://www.mediawiki.org/wiki/Extension:MassMessage/API)
* [python-mwapi documentation](http://pythonhosted.org/mwapi/)
 * [OAuth demo](https://github.com/mediawiki-utilities/python-mwapi/blob/master/demo_mwoauth.py)

In [227]:
import mwapi
import datetime as dt
from requests_oauthlib import OAuth1
import secrets.oauth as oauth_cfg
import re

In [228]:
real_strata = pd.read_csv("sampled-strata.tsv", sep = "\t")
real_strata.head()

,proj_group,edit_bin,sample_size,sampled_users,page_title,survey_url,preview_url
0,arwiki,"[10, 30)",26,"[user(user='Michael M. Markos', proj_domain='a...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
1,arwiki,"[30, 150)",94,"[user(user='المسرحي', proj_domain='ar.wikipedi...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
2,arwiki,"[150, 600)",96,"[user(user='Budur Subaie', proj_domain='ar.wik...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
3,arwiki,"[600, 1200)",56,"[user(user='Ahmed El-Farash', proj_domain='ar....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
4,arwiki,"[1200, 3500)",68,"[user(user='Ibrahim Old', proj_domain='ar.wiki...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...


In [229]:
real_strata.sample(n = 10)

,proj_group,edit_bin,sample_size,sampled_users,page_title,survey_url,preview_url
51,itwiki,"[600, 1200)",55,"[user(user='APALERMO90', proj_domain='it.wikip...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
92,weur_wps,"[150, 600)",434,"[user(user='Drefer', proj_domain='fi.wikipedia...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
106,zhwiki,"[1200, 3500)",215,"[user(user='Wanchens', proj_domain='zh.wikiped...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
38,eswiki,"[150, 600)",131,"[user(user='Resgo27', proj_domain='es.wikipedi...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
24,dewiki,"[10, 30)",42,"[user(user='Güsen17', proj_domain='de.wikipedi...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
3,arwiki,"[600, 1200)",56,"[user(user='Ahmed El-Farash', proj_domain='ar....",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
60,meaf_wps,"[10, 30)",27,"[user(user='Mahdi.joodi', proj_domain='fa.wiki...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
85,ruwiki,"[30, 150)",114,"[user(user='AlfaDesign', proj_domain='ru.wikip...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
72,other,"[10, 30)",29,"[user(user='Karttapallo1', proj_domain='fi.wik...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...
65,meaf_wps,"[3500, 1100000)",73,"[user(user='Behnam mancini', proj_domain='fa.w...",Community Engagement Insights/MassMessages/Lis...,https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...


# Configuration

In [193]:
# A descriptive user agent for our API requests
USER_AGENT = (
    "CE Insights survey bot -- " +
    "https://github.com/wikimedia-research/Community-Engagement-Insights-sampling"
)

# A MassMessage list for testing. 
TEST_LIST = "User:Neil P. Quinn-WMF/MassMessage test list"

Message assembly code:
```
{{#invoke:
Assemble multilingual message|
assembleMessage|
marker=ce-insights-content|
page=Community Engagement Insights/MassMessages/First message|
ar|de|es|fr|it|ja|nl|pl|pt|ru|uk|zh
}}
```

In [194]:
# The page with the message content you actually want to send,
# assembled as described at https://meta.wikimedia.org/wiki/Newsletters/Translation#Regular_process
MESSAGE_PAGE = "User:Neil P. Quinn-WMF/2018 survey invitation"

# Set up connection

Use OAuth to authenticate as an [owner-only consumer](https://www.mediawiki.org/wiki/OAuth/Owner-only_consumers).

In [191]:
sess = mwapi.Session("https://meta.wikimedia.org", user_agent = USER_AGENT )

In [192]:
auth = OAuth1(
    oauth_cfg.consumer_token,
    oauth_cfg.consumer_secret,
    oauth_cfg.access_token,
    oauth_cfg.access_secret
)

In [195]:
def get_token():
    resp = sess.get(
        action="query", 
        meta="tokens", 
        type="csrf", 
        auth = auth
    )
    
    return resp["query"]["tokens"]["csrftoken"]

In [218]:
def api_get(*args, **kwargs):
    return sess.get(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        **kwargs
    )
    
def api_post(*args, **kwargs):
    return sess.post(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        token = get_token(),
        **kwargs
    )

# Check API authentication 

In [219]:
api_get(
    action = "query",
    meta = "userinfo"
)

{'batchcomplete': True,
 'query': {'userinfo': {'id': 19115528, 'name': 'WMF Surveys'}}}

# Assemble message

In [222]:
msg = api_get(
    action = "parse",
    page = MESSAGE_PAGE
)["parse"]["text"]

In [224]:
# Replace links
def add_link(msg, link):
    return re.sub(
        r"https://www.example.com",
        link,
        msg
    )

# Set up messaging

In [238]:
def send_message(target_list, subj, msg):
    api_post(
        action = "massmessage",
        spamlist = target_list,
        subject = subj,
        message = msg
    )


def message_frame(frame):
    for row in frame.itertuples():
        message_list = row.page_title
        survey_url = row.survey_url
        preview_url = row.preview_url
        
        linked_msg = add_link(msg, survey_url)
        
        print(message_list)
        print(linked_msg)

# Test messaging

## Basic test

In [ ]:
send_message(
    TEST_LIST,
    "Testing MassMessage",
    "Did this work properly? ~~~~"
)

## Test with full message

In [65]:
def massMessageAll():
    print_err("messaging 1")
    print_err("messaging 2")

In [66]:
massMessageAll()

messaging 1
messaging 2


In [197]:
# Run this cell and follow the prompt to unlock sending code for 30 seconds
ipyd.HTML("""
<script type="text/Javascript">
var kernel = IPython.notebook.kernel;

if (prompt("The code below will send talk pages messages to thousands of editors. Type 'run' to unlock it")) {
    kernel.execute("allow_run = dt.datetime.now()")
};
</script
""")

In [198]:
# If the cell above has been run within the past 30 seconds, send the messages
if (dt.datetime.now() - allow_run) < datetime.timedelta(seconds = 30):
    print("Sending!")
    print_err("1")
    print_err("2")
else:
    print_err(
        "This code will send talk page messages to thousands of editors.", 
        "If you're certain you want to do this, run the cell above to unlock it.",
        sep = "\n"
    )

Sending!


1
2
